# `gradelib` demo

The below demonstrates how to use `gradelib`, as well as the features available for overvieweing the grade distribution in a Jupyter notebook.

In [5]:
import gradelib
from gradelib.policies import exceptions, lates, drops

# combine grades from Gradescope and Canvas
gb = gradelib.combine_gradebooks([
    gradelib.io.gradescope.read('./gradescope.csv'),
    gradelib.io.canvas.read('./canvas.csv')
])

HOMEWORKS = gb.assignments.starting_with("home")
LABS = gb.assignments.starting_with("lab")

# group the assignments and determine their weight in the overall score calculation
gb.grading_groups = {
    'homeworks': (HOMEWORKS, .25),
    'labs': (LABS, .25),
    'midterm exam': .25,
    'final exam': .25
}

# handle exceptions
exceptions.make_exceptions(
    gb,
    "Justin",
    [
        exceptions.Drop("lab 01", reason="illness"),
        exceptions.Replace("homework 01", with_="homework 02", reason="added late")
    ]
)

# apply grading policies
lates.penalize(gb, policy=lates.Forgive(3), within=HOMEWORKS + LABS)
drops.drop_most_favorable(gb, 1, within=HOMEWORKS)
drops.drop_most_favorable(gb, 1, within=LABS)

# find robust letter grade cutoffs by clustering grades
gb.scale = gradelib.scales.find_robust_scale(gb.overall_score)

In [6]:
gradelib.overview(gb)

Letter,A+,A,A-,B+,B,B-,C+,C,C-,D,F
Frequency,1,1,2,0,0,0,0,0,0,0,0


Loading BokehJS ...

,Frequency
Number of Lates,
1,1
2,2
5,1


,homeworks,labs,midterm exam,final exam,overall score,letter,rank,percentile
<Fitzgerald Zelda>,0.991071,1.000000,0.916667,0.966667,0.968601,A+,1,1.00
<Obama Barack>,0.846774,1.000000,1.000000,0.966667,0.953360,A,2,0.75
<Eldridge Justin>,0.793491,0.914062,0.958333,1.000000,0.916472,A-,3,0.50
<Else Someone>,0.794643,0.920290,0.937500,0.933333,0.896442,A-,4,0.25
